In [1]:
import scanpy as sc
import pandas as pd
from xopen import xopen
import pysam

In [43]:
df = pd.read_csv("/SGRNJ06/randd/PROJECT/scATAC/20230529_3bp_959595_H_M_jiace_sc/A0508_4_mixCCRFtesticles_3rd959595_D15_T7100_612/03.atac/A0508_4_mixCCRFtesticles_3rd959595_D15_T7100_612/outs/singlecell.csv")

In [44]:
df["barcode"] = df["barcode"].apply(lambda x: x.split('-')[0])

In [45]:
df_h = df[df["is_GRCh38_cell_barcode"]==1]
df_m = df[df["is_mm10_cell_barcode"]==1]

In [46]:
h_bc = set(df_h.barcode)
m_bc = set(df_m.barcode)
doublet_bc = h_bc & m_bc

In [60]:
"AAATGAGAGCAACTGG" in h_bc

False

In [ ]:
doublet_bc

In [48]:
out_fq1 = xopen("/SGRNJ06/randd/USER/cjj/celedev/atac/20230531del_doublet/A0508_4_mixCCRFtesticles_3rd959595_D15_T7100_612/deldoublets_S1_L001_R1_001.fastq.gz", 'w')
out_fq2 = xopen("/SGRNJ06/randd/USER/cjj/celedev/atac/20230531del_doublet/A0508_4_mixCCRFtesticles_3rd959595_D15_T7100_612/deldoublets_S1_L001_R2_001.fastq.gz", 'w')
out_fq3 = xopen("/SGRNJ06/randd/USER/cjj/celedev/atac/20230531del_doublet/A0508_4_mixCCRFtesticles_3rd959595_D15_T7100_612/deldoublets_S1_L001_R3_001.fastq.gz", 'w')

In [49]:
with pysam.FastxFile("/SGRNJ06/randd/USER/cjj/celedev/atac/20230531del_doublet/A0508_4_mixCCRFtesticles_3rd959595_D15_T7100_612/A0508_4_mixCCRFtesticles_3rd959595_D15_T7100_612_S1_L001_R1_001.fastq.gz", persist=False) as fq1, \
        pysam.FastxFile("/SGRNJ06/randd/USER/cjj/celedev/atac/20230531del_doublet/A0508_4_mixCCRFtesticles_3rd959595_D15_T7100_612/A0508_4_mixCCRFtesticles_3rd959595_D15_T7100_612_S1_L001_R2_001.fastq.gz", persist=False) as fq2, \
            pysam.FastxFile("/SGRNJ06/randd/USER/cjj/celedev/atac/20230531del_doublet/A0508_4_mixCCRFtesticles_3rd959595_D15_T7100_612/A0508_4_mixCCRFtesticles_3rd959595_D15_T7100_612_S1_L001_R3_001.fastq.gz", persist=False) as fq3:
    for entry1, entry2, entry3 in zip(fq1, fq2, fq3):
        header1, seq1, qual1 = entry1.name, entry1.sequence, entry1.quality
        header2, seq2, qual2 = entry2.name, entry2.sequence, entry2.quality
        header3, seq3, qual3 = entry3.name, entry3.sequence, entry3.quality
        
        if seq2 not in doublet_bc:
            out_fq1.write(f'@{header1}\n{seq1}\n+\n{qual1}\n')
            out_fq2.write(f'@{header2}\n{seq2}\n+\n{qual2}\n')
            out_fq3.write(f'@{header3}\n{seq3}\n+\n{qual3}\n')
        

In [50]:
out_fq1.close()
out_fq2.close()
out_fq3.close()